In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Khởi tạo tokenizer và mô hình PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base")

# Hàm để chia tài liệu thành các đoạn nhỏ hơn
def chunk_document(text, max_length=100):
    tokens = tokenizer.tokenize(text)
    chunks = [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]
    return chunks

# Hàm để mã hóa đoạn văn bản
def encode_chunk(chunk):
    # Nếu chunk trống, trả về một vector không (zero vector) hoặc bỏ qua
    if len(chunk) == 0:
        return None
    inputs = tokenizer(" ".join(chunk), return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    chunk_vector = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    return chunk_vector

# Hàm để mã hóa toàn bộ tài liệu
def encode_long_text(text):
    chunks = chunk_document(text)
    chunk_vectors = []
    for chunk in chunks:
        chunk_vector = encode_chunk(chunk)
        if chunk_vector is not None:  # Chỉ thêm các vector hợp lệ
            chunk_vectors.append(chunk_vector)
    return chunk_vectors

# Ví dụ tài liệu dài
long_document = "sạt lở đất ở Tà Xùa, Sơn La ngày 10/8. Ảnh: Ban chỉ huy quân sự huyện Bắc YênTrung tâm Dự báo Khí tượng Thủy văn quốc gia nhận định từ nay đến ngày 17/8, miền Bắc sẽ mưa lớn, cao điểm mưa 60-120 mm, có nơi trên 250 mm tập trung từ 11 đến 13/8. Lượng mưa lớn, tập trung vào chiều tối và đêm, trên phạm vi hẹp (ở không gian cấp tỉnh) nên khả năng cao gây lũ quét, sạt lở đất, ngập úng tại các khu vực trũng thấp, đặc biệt ở các tỉnh vùng núi phía Bắc.Trang Accuweather của Mỹ dự báo Hà Nội tuần tới phổ biến 26-32 độ C, riêng thứ năm nhiệt độ cao nhất trong ngày lên 34 độ. Điểm cao trên 1.500 m so với mực nước biển như Sa Pa (Lào Cai) nhiệt độ trong tuần 17-26 độ C.Miền Trung các tỉnh Thanh Hóa, Nghệ An chịu ảnh hưởng của rãnh áp thấp ở phía bắc nên đêm nay và chiều tối mai có mưa rào, giông. Vùng nắng nóng thu hẹp vào các tỉnh Hà Tĩnh - Bình Thuận với nhiệt độ cao nhất ngày phổ biến 35-37 độ, có nơi trên 37 độ C.Nam Bộ và Tây Nguyên từ nay đến 13/8 có mưa rào, giông về chiều tối. Các ngày 14-17/8, gió mùa tây nam suy yếu, mưa tiếp tục giảm.Nhiệt độ cao nhất ở Nam Bộ phổ biến 32-34 độ, Tây Nguyên 29-32 độ C. Từ ngày 14/7, nhiệt độ Nam Bộ tăng lên 33-35 độ, miền Đông Nam Bộ có nắng nóng cục bộ, Tây Nguyên 30-33 độ C.Miền Bắc vừa trải qua gần một tuần nắng nóng nhưng các tỉnh miền núi vẫn mưa rào, giông về chiều tối và đêm. Độ ẩm trong đất ở nhiều khu vực đã bão hòa nên nguy cơ sạt lở đất rất cao.Bộ Nông nghiệp và Phát triển nông thôn hôm qua tiếp tục yêu cầu UBND các tỉnh miền Bắc theo dõi diễn biến thời tiết, tổ chức kiểm tra khu vực có nguy cơ lũ quét, sạt lở. Các địa phương tổ chức canh gác ở ngầm tràn, khu vực đã hoặc có nguy cơ sạt lở, không cho người qua lại; sẵn sàng lực lượng, phương tiện cứu hộ, cứu nạn khi có yêu cầu."  # Ví dụ dài

# Mã hóa tài liệu
document_vectors = encode_long_text(long_document)

# Kiểm tra số lượng đoạn đã mã hóa
print(f"Số lượng đoạn đã mã hóa: {len(document_vectors)}")